## Data For Report
The data that we have chosen to zero in on are going to be those that would directly impact the severity of a collision. The lat/long of a collision has no correlation to the severity of the accident, so, for the purposes of our analysis, we can drop it as part of the model. As a part of the preparation, we need to sift through the data in order to see what aspects are truly relevant.  

We will create a correlation matrix using all of the data that we have, to see if there is a baseline of correlation from one variable to the next. The goal will be to first identify which items have high correlation to severity. Then, the next aspect is to see if there is redundant data. We don't need two independent variables that are highly correlated to both be part of the calculation for severity, since it will bias the model.  

I already notice that with my data description, I am focusing too much on the cleaning aspect. Invariably, the cleaning is directly tied to the understanding of the data therein, so I will continue to conduct any and all cleaning in this file, and will draw conclusions about which data I will retain and which data I will drop. The information will be contained on the top of this document, here, and will be contained below, at the end of the cleaning process. The final step will include outputting the cleaned data as a csv for further use in a different file.  

I will note my logic for dropping columns if I feel I need further justification. Given that 'SEVERITYDESC' has 100% correlation to 'SEVERITYCODE', I will drop this column, since it is a perfect, and hindsight based predictor of severity, and removes any predictability power from the model entirely.

In [311]:
import pandas as pd
import numpy as np

collision_data = pd.read_csv("Data-Collisions.csv")

In [312]:
collision_data.describe()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,INTKEY,SEVERITYCODE.1,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,SDOT_COLCODE,SDOTCOLNUM,SEGLANEKEY,CROSSWALKKEY
count,194673.000000,189339.000000,189339.000000,194673.000000,194673.000000,194673.000000,65070.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,1.149360e+05,194673.000000,1.946730e+05
mean,1.298901,-122.330518,47.619543,108479.364930,141091.456350,141298.811381,37558.450576,1.298901,2.444427,0.037139,0.028391,1.920780,13.867768,7.972521e+06,269.401114,9.782452e+03
std,0.457778,0.029976,0.056157,62649.722558,86634.402737,86986.542110,51745.990273,0.457778,1.345929,0.198150,0.167413,0.631047,6.868755,2.553533e+06,3315.776055,7.226926e+04
min,1.000000,-122.419091,47.495573,1.000000,1001.000000,1001.000000,23807.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.007024e+06,0.000000,0.000000e+00
25%,1.000000,-122.348673,47.575956,54267.000000,70383.000000,70383.000000,28667.000000,1.000000,2.000000,0.000000,0.000000,2.000000,11.000000,6.040015e+06,0.000000,0.000000e+00
50%,1.000000,-122.330224,47.615369,106912.000000,123363.000000,123363.000000,29973.000000,1.000000,2.000000,0.000000,0.000000,2.000000,13.000000,8.023022e+06,0.000000,0.000000e+00
75%,2.000000,-122.311937,47.663664,162272.000000,203319.000000,203459.000000,33973.000000,2.000000,3.000000,0.000000,0.000000,2.000000,14.000000,1.015501e+07,0.000000,0.000000e+00
max,2.000000,-122.238949,47.734142,219547.000000,331454.000000,332954.000000,757580.000000,2.000000,81.000000,6.000000,2.000000,12.000000,69.000000,1.307202e+07,525241.000000,5.239700e+06


In [313]:
collision_data.drop(collision_data[collision_data.EXCEPTRSNCODE == 'NEI'].index, inplace=True)
collision_data.drop(columns=['EXCEPTRSNCODE', 'EXCEPTRSNDESC'], inplace=True)

In [314]:
# INATTENTIONIND SET Y = 1 BLANK = 0
# UNDERINFL SET Y, = 1, N = 0, BLANK = 0
# PEDROWNOTGRNT SET Y = 1, BLANK = 0
# SPEEDING SET Y = 1, BLANK = 0
# HITPARKEDCAR SET Y = 1, N = 1
collision_data['INATTENTIONIND'] = np.where((collision_data.INATTENTIONIND == 'Y'), 1, 0)
collision_data['PEDROWNOTGRNT'] = np.where((collision_data.PEDROWNOTGRNT == 'Y'), 1, 0)
collision_data['SPEEDING'] = np.where((collision_data.SPEEDING == 'Y'), 1, 0)
collision_data['HITPARKEDCAR'] = np.where((collision_data.HITPARKEDCAR == 'Y'), 1, 0)
collision_data['UNDERINFL'] =   np.where(((collision_data.UNDERINFL == 'Y') | (collision_data.UNDERINFL == '1')), 1, 0)

In [315]:
#Now to convert the datetime to just the time of the day, and place them into buckets. these buckets will occur by just returning the hour of the day, and using that hour as the bucket for our incident.
# pd.to_datetime(collision_data['INCDTTM'])
collision_data['HOUROFDAY'] = pd.to_datetime(collision_data['INCDTTM']).dt.hour



In [316]:
collision_data['HOUROFDAY']

0         14
1         18
2         10
3          9
4          8
          ..
194668     8
194669     9
194670     9
194671    16
194672    15
Name: HOUROFDAY, Length: 189035, dtype: int64

In [317]:
collision_data_relevant = collision_data.drop(columns=['X', 'Y', 'OBJECTID', 'INCKEY','INCDATE', 'COLDETKEY', 'INTKEY', 'STATUS', 'LOCATION', 'STATUS', 'REPORTNO', 'SEVERITYCODE.1', 'SDOT_COLDESC', 'SDOTCOLNUM', 'ST_COLCODE','INCDTTM','SEVERITYDESC','SEGLANEKEY']).dropna()

In [318]:
#Now to convert our categorical values into numeric values, so that our tree can accurately interpret them. We will utilize a for-loop in order to rapidly do this
labels_list = []
for i in ['ADDRTYPE','COLLISIONTYPE','JUNCTIONTYPE','WEATHER','ROADCOND','LIGHTCOND','ST_COLDESC']:
    labels = collision_data_relevant[i].astype('category').cat.categories.tolist()
    replace_map_comp = {i : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}
    collision_data_relevant.replace(replace_map_comp, inplace=True)
    labels_list.append(i)
    print(replace_map_comp)

{'ADDRTYPE': {'Alley': 1, 'Block': 2, 'Intersection': 3}}
{'COLLISIONTYPE': {'Angles': 1, 'Cycles': 2, 'Head On': 3, 'Left Turn': 4, 'Other': 5, 'Parked Car': 6, 'Pedestrian': 7, 'Rear Ended': 8, 'Right Turn': 9, 'Sideswipe': 10}}
{'JUNCTIONTYPE': {'At Intersection (but not related to intersection)': 1, 'At Intersection (intersection related)': 2, 'Driveway Junction': 3, 'Mid-Block (but intersection related)': 4, 'Mid-Block (not related to intersection)': 5, 'Ramp Junction': 6, 'Unknown': 7}}
{'WEATHER': {'Blowing Sand/Dirt': 1, 'Clear': 2, 'Fog/Smog/Smoke': 3, 'Other': 4, 'Overcast': 5, 'Partly Cloudy': 6, 'Raining': 7, 'Severe Crosswind': 8, 'Sleet/Hail/Freezing Rain': 9, 'Snowing': 10, 'Unknown': 11}}
{'ROADCOND': {'Dry': 1, 'Ice': 2, 'Oil': 3, 'Other': 4, 'Sand/Mud/Dirt': 5, 'Snow/Slush': 6, 'Standing Water': 7, 'Unknown': 8, 'Wet': 9}}
{'LIGHTCOND': {'Dark - No Street Lights': 1, 'Dark - Street Lights Off': 2, 'Dark - Street Lights On': 3, 'Dark - Unknown Lighting': 4, 'Dawn': 5, 

In [319]:
collision_data_relevant.reset_index(inplace=True, drop=True)
collision_data_relevant

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLDESC,CROSSWALKKEY,HITPARKEDCAR,HOUROFDAY
0,2,3,1,2,0,0,2,2,11,0,0,5,9,6,0,0,5,0,0,14
1,1,2,10,2,0,0,2,5,16,0,0,7,9,3,0,0,17,0,0,18
2,1,2,6,4,0,0,3,5,14,0,0,5,1,6,0,0,28,0,0,10
3,1,2,5,3,0,0,3,5,11,0,0,2,1,6,0,0,15,0,0,9
4,2,3,1,2,0,0,2,2,11,0,0,7,9,6,0,0,5,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181061,2,2,3,3,0,0,2,5,11,0,0,2,1,6,0,0,11,0,0,8
181062,1,2,8,2,0,0,2,5,14,1,0,7,9,6,0,0,16,0,0,9
181063,2,3,4,3,0,0,2,2,11,0,0,2,1,6,0,0,13,0,0,9
181064,2,3,2,2,0,1,1,2,51,0,0,2,1,7,0,0,53,0,0,16


In [320]:
collision_data_relevant.corr(method='pearson', min_periods=1)

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLDESC,CROSSWALKKEY,HITPARKEDCAR,HOUROFDAY
SEVERITYCODE,1.000000,0.191201,-0.126293,0.123792,0.244164,0.213477,-0.079848,-0.198740,0.161704,0.040400,0.039705,-0.084248,-0.033417,-0.036926,0.206038,0.033914,0.099013,0.172777,-0.087120,0.031489
ADDRTYPE,0.191201,1.000000,-0.482232,0.059587,0.143217,0.082480,-0.090043,-0.919140,-0.068789,-0.083471,-0.047527,-0.069945,-0.018598,-0.033325,0.155258,-0.065089,-0.168275,0.176120,-0.114501,0.041482
COLLISIONTYPE,-0.126293,-0.482232,1.000000,0.015785,0.093463,-0.211996,0.104975,0.482510,0.014042,0.122779,0.005498,0.018963,-0.006616,0.025141,-0.020564,-0.002296,0.361238,0.033517,0.032658,-0.005694
PERSONCOUNT,0.123792,0.059587,0.015785,1.000000,-0.026629,-0.042534,0.399715,-0.069830,-0.177460,0.071111,0.018098,-0.050895,-0.023667,-0.027323,-0.031731,-0.007835,-0.067916,-0.034363,-0.042441,0.030907
PEDCOUNT,0.244164,0.143217,0.093463,-0.026629,1.000000,-0.018562,-0.315981,-0.130424,0.283327,-0.008240,0.014795,-0.004351,0.009657,-0.035135,0.496801,-0.035003,0.564382,0.568736,-0.031187,0.025825
PEDCYLCOUNT,0.213477,0.082480,-0.211996,-0.042534,-0.018562,1.000000,-0.306282,-0.087600,0.418538,0.001044,-0.018475,-0.050059,-0.047357,0.019001,0.325585,-0.022378,0.357401,0.109444,-0.027379,0.022931
VEHCOUNT,-0.079848,-0.090043,0.104975,0.399715,-0.315981,-0.306282,1.000000,0.088328,-0.507194,0.051240,-0.011347,-0.012246,-0.017825,0.034697,-0.277556,-0.048845,-0.216368,-0.236850,0.073987,0.010129
JUNCTIONTYPE,-0.198740,-0.919140,0.482510,-0.069830,-0.130424,-0.087600,0.088328,1.000000,0.080161,0.072035,0.057061,0.081328,0.025302,0.026615,-0.153624,0.067151,0.172640,-0.160084,0.137860,-0.033514
SDOT_COLCODE,0.161704,-0.068789,0.014042,-0.177460,0.283327,0.418538,-0.507194,0.080161,1.000000,0.010037,0.103097,-0.025788,0.003568,-0.104742,0.255301,0.146457,0.281607,0.205167,-0.064020,-0.035238
INATTENTIONIND,0.040400,-0.083471,0.122779,0.071111,-0.008240,0.001044,0.051240,0.072035,0.010037,1.000000,-0.030593,-0.074548,-0.050762,0.011343,-0.030380,-0.054071,0.024330,-0.004677,0.019401,0.026332


In [321]:
collision_data_relevant.to_csv('Cleaned-Data-Collisions.csv')

### Data Cleaning Analysis
#### Looking closer at the remaining variables that we will use for our model.
We have accomiplished our stated goals for cleaning the data. As we build and test our model, we will return to this file to adjust what data we want to utilize, in order to maximize the performance of the model.  
We eliminated columns that did not seem to have any bearing on the outcome; latitude/longitude coordinates were irrelevant, as well as the primary/secondary keys that each accident was uniquely assigned. Any component of the analysis that was unique to the incident and was not repeated with any level of meaning was dropped from the dataset. A good example of this is the date, since the date of the incident will bear very little meaning on the severity of the incident itself.  
Next, we identified categorical variables that used strings as their values. We chose to include the string version instead of the numeric version in order to create the appropriate list of references for our dataset, thereby making the analysis component much smoother. With the string versions included, and the numeric versions dropped, we then ran a quick for loop to apply integer values to the categorical variables, and replaced the strings with the new numbers. Finally, there were some columns in the dataset that has many "blank" values. An assumption was made here, that will need to be validated in the analysis section. We assumed that any blank value was a "null" value, and that the officer/data entrist simply neglected to fill in the information. The assumption, while maybe flawed, is that "importance" of a variable would have incentivized a data entrist to record in the affirmative, such as in the case of a driver under the influence. If the driver was in indeed under the influence, then we can assume that the entrist would have included it in the data. If the driver was not, however, then the entrist may have simply neglected to record the information.  
Now, with our data in numerical integer form, we created a correlation matrix. While some correlation values seem small and otherwise insignificant, we will retain them, as they still might add to our accuracy, and not detract.

